In [2]:
tf.test.is_gpu_available()
tf.config.list_physical_devices('GPU')

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
blah = 10000

In [1]:
from importlib import reload
import numpy as np
import os
import copy
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import matplotlib.pyplot as plt
from scipy.sparse import csc_matrix, csr_matrix
import pickle
import json
import pretty_midi
import sys
from collections import namedtuple
import timeit
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.callbacks import LearningRateScheduler
# import my modules
import src.midi_utils as midi_utils
import src.data as data
import src.models as models
import src.ml_classes as ml_classes
import src.exp_utils as exp_utils
import src.losses as losses


In [ ]:

# seem to need this to use my custom loss function, see here: https://github.com/tensorflow/tensorflow/issues/34944
# last answer might fix it: https://stackoverflow.com/questions/57704771/inputs-to-eager-execution-function-cannot-be-keras-symbolic-tensors
# the trick is for the step that defines the loss fnc to return a symbolic tensor, rather than returning another function which uses a symbolic tensor
tf.compat.v1.disable_eager_execution()
# alternatively, could do something like this?
# https://github.com/Douboo/tf_env_debug/blob/master/custom_layers_and_model_subclassing_API.ipynb

# data params
model_inputs = ['H', 'V_mean']
model_outputs = ['H', 'V']
seq_length = 64
sub_beats = 2
use_base_key = True
transpose = False
st = 0
nth_file = 10
vel_cutoff = 4
data_folder_prefix = '_8'

##### Model Config ####
### general network params
hierarchical = False
variational = True
latent_size = 10
hidden_state = 100
dense_size = 100
dense_layers = 2
recurrent_dropout=0.0

### encoder params
encoder_lstms = 2
z_activation = None
conv = False
# pitch_stride = 6
# conv = {'F_n': [32, 32, 48, 48, 48, 24], # number of filters
#         'F_s': [(8,12), (4,4), (4,4), (4,4), (4,4), (4,4)], # size of filters
#         'strides': [(1, pitch_stride), (1, 1), (2, 1), (2,1), (2,1), (2,2)],  # strides
#         'batch_norm': True # apply batch norm after each conv operation (after activation)
#         }


### sampling params... if applicable.
epsilon_std=1

### decoder params
decoder_lstms=2
# ar_inputs only works as parameter for non hierarchical graph, currently
ar_inputs = None
conductors=2
conductor_steps= int(seq_length/16)
conductor_state_size=None # none => same as decoder
initial_state_from_dense=True
initial_state_activation='tanh'

##### Training Config ####
batch_size = 64
lr = 0.001
lr_decay_rate = 0.2**(1/1500)
min_lr = 0.00005
epochs = 2
monitor = 'loss'
loss_weights = [1, 3]
clipvalue = 1
loss = losses.vae_custom_loss
# loss = 'categorical_crossentropy'
metrics = ['accuracy', 'categorical_crossentropy']

# musicvae used 48 free bits for 2-bars, 256 for 16 bars (see https://arxiv.org/pdf/1803.05428.pdf)
# Variational specific parameters
max_beta = 0.05
beta_rate = 0.2**(1/1000) # at 1000 epochs, we want (1 - something) * max_beta
free_bits=0
kl_weight = 1

#other
continue_run = None
log_tensorboard = False
ar_inc_batch_shape = False # sometimes needed to make training work...
     

blah = blah + 1
no, path = exp_utils.set_up_path(blah)


# get training data
assert seq_length % 4 == 0, 'Sequence length must be divisible by 4'
model_datas_train, seconds = data.folder2examples('training_data/midi_train' + data_folder_prefix, sparse=True, use_base_key=use_base_key, beats_per_ex=int(seq_length / sub_beats), nth_file=nth_file, vel_cutoff=vel_cutoff, sub_beats=sub_beats)
model_datas_val, seconds = data.folder2examples('training_data/midi_val' + data_folder_prefix, sparse=True, use_base_key=use_base_key, beats_per_ex=int(seq_length / sub_beats), sub_beats=sub_beats)

model_input_reqs, model_output_reqs = models.get_model_reqs(model_inputs, model_outputs)

callbacks = []

if log_tensorboard:
    callbacks.append(tf.keras.callbacks.TensorBoard(log_dir='experiments/tb/', histogram_freq = 1))

# model kwargs - for the encoder/decoder builder functions, make a dictionary to pass as kwargs
model_kwargs = {# general model parameters
                'recurrent_dropout':recurrent_dropout,
                'hidden_state':hidden_state,
                'dense_size':dense_size,
                'seq_length':seq_length,

                # encoder parameters
                'latent_size':latent_size,
                'z_activation':z_activation,
                'variational':variational,
                'conv':conv,

                # decoder parameters
                'ar_inputs':ar_inputs, 
                'decoder_lstms':decoder_lstms,
                'batch_size':batch_size, # not used in encoder, currently...
                'initial_state_from_dense':initial_state_from_dense,
                'initial_state_activation':initial_state_activation,
                'ar_inc_batch_shape':ar_inc_batch_shape,
                'conv':conv,
                # conductor configuration (only used if hierarchical)
                'conductor_state_size':conductor_state_size, # none => same as decoder
                'conductors':conductors,
                'conductor_steps':conductor_steps,
                }
# if variational, z will be a list of [[means], [stds]]
build_encoder_graph = models.create_LSTMencoder_graph
z, model_inputs_tf = build_encoder_graph(model_input_reqs, **model_kwargs)

if variational:
    beta_fn = exp_utils.beta_fn2(beta_rate, max_beta)
    loss_for_train, beta_cb = loss(z, beta_fn, free_bits=free_bits, kl_weight=kl_weight)
    sampling_fn = models.sampling(batch_size, epsilon_std=epsilon_std)
    # z_input is the tensor that will be passed into the decoder
    z_input = layers.Lambda(sampling_fn)(z)
    if not isinstance(beta_fn, (int, float)):
        callbacks.append(beta_cb)
else:
    loss_for_train = loss
    z_input = z

if hierarchical:
    build_decoder_graph = models.create_hierarchical_decoder_graph
else:
    build_decoder_graph =models.create_LSTMdecoder_graph_ar

pred, ar_inputs_tf = build_decoder_graph(z_input, model_output_reqs, **model_kwargs)
autoencoder = tf.keras.Model(inputs=model_inputs_tf + ar_inputs_tf, outputs=pred, name=f'autoencoder')
autoencoder.summary()

if continue_run != None:
    autoencoder.load_weights(f'experiments/run_{continue_run}/{continue_run}_best_train_weights.hdf5')



# save a plot of the model
# tf.keras.utils.plot_model(seq_model, to_file=f'{path}model_plot.png')

dg = ml_classes.ModelDataGenerator([md for md in model_datas_train.values()],
                                    [model_in.name for model_in in model_input_reqs],
                                    [model_out.name for model_out in model_output_reqs],
                                    t_force=True, batch_size = batch_size, seq_length=seq_length)

dg_val = ml_classes.ModelDataGenerator([md for md in model_datas_val.values()],
                                    [model_in.name for model_in in model_input_reqs],
                                    [model_out.name for model_out in model_output_reqs],
                                    t_force=True, batch_size = batch_size, seq_length=seq_length)

opt = tf.keras.optimizers.Adam(learning_rate=lr, clipvalue=clipvalue)
autoencoder.compile(optimizer=opt, loss=loss_for_train, metrics=metrics, loss_weights=loss_weights)
# history = autoencoder.fit(dg, validation_data=dg_val, epochs=epochs, callbacks=callbacks, verbose=2)


In [ ]:
# import modules, including a reload statement so that they can be reimported after a change to the methods 
import src.midi_utils as midi_utils
reload(midi_utils)

import src.data as data
reload(data)

import src.models as models
reload(models)

import src.ml_classes as ml_classes
reload(ml_classes)

In [43]:
### Make some predictions ###
# load best weights
# models.load_weights_safe(autoencoder, path + f'{no}_best_train_weights.hdf5', by_name=False)
# get some random examples from the validation data
random_examples, idx = data.n_rand_examples(model_datas_val, n=batch_size)

# currently, prediction is broken for my variational models
# if variational, then for now need to circumvent autoencoder.predict()
if variational:
    #grab encoder layers from above, make a model
#     z, model_inputs_tf = build_encoder_graph(model_input_reqs, **model_kwargs)
#     encoder = tf.keras.Model(inputs=model_inputs_tf, outputs=z, name=f'encoder')
    
    # create a new decoder
    z_in = tf.keras.Input(shape=(latent_size,), name='z')
    model_kwargs['ar_inc_batch_shape'] = False
    pred, ar_inputs_tf = build_decoder_graph(z_in, model_output_reqs, **model_kwargs)
    decoder = tf.keras.Model(inputs=[z_in] + ar_inputs_tf, outputs=pred, name=f'decoder')

    # load weights for decoder
    # models.load_weights_safe(decoder, path + f'{no}_best_train_weights.hdf5', by_name=True)

    in_dict = dg_val.__getitem__(0)[0]

    ### predict
    # get paramerterization of latent space
    zp_param = encoder.predict(in_dict)
    # generate random values
    zp_latent = sampling_fn(zp_param)
    with tf.compat.v1.Session():
        in_dict['z'] = zp_latent.eval()
#         in_dict['z'] = np.zeros((64,10))
    pred = decoder.predict(in_dict)

else:
    pred = autoencoder.predict(random_examples)

# find axis that corresponds to velocity
v_index = np.where(np.array(autoencoder.output_names) == 'V_out')[0][0]
print('velocity index:', v_index)
model_datas_pred, _ = data.folder2examples('training_data/midi_val' + data_folder_prefix, sparse=False, use_base_key=use_base_key, beats_per_ex=int(seq_length / sub_beats), sub_beats=sub_beats)
model_datas = copy.deepcopy(model_datas_pred)
model_datas_pred['V'].data[idx,...] = np.array(pred)[v_index,:,:,:]
os.mkdir(path + 'midi/')
for i in idx:
    pm_original = data.examples2pm(model_datas, i, sub_beats=sub_beats)
    pm_pred = data.examples2pm(model_datas_pred, i, sub_beats=sub_beats)
    pm_original.write(path + 'midi/' + f'ex{i}original.mid')
    pm_pred.write(path + 'midi/' + f'ex{i}prediction_teacher_forced.mid')

  8%|▊         | 1/12 [00:00<00:02,  5.17it/s]

velocity index: 1


100%|██████████| 12/12 [00:03<00:00,  3.28it/s]


00h 37m 53s of data
created model data H :    (64, 88) data shape,     122 training examples
created model data O :    (64, 88) data shape,     122 training examples
created model data V :    (64, 88) data shape,     122 training examples
created model data R :    (64, 88) data shape,     122 training examples
created model data S :    (64, 2) data shape,     122 training examples
created model data key :    (12,) data shape,     122 training examples
created model data tempo :    (1,) data shape,     122 training examples
created model data V_mean :    (1,) data shape,     122 training examples
example 91 chosen
example 91 chosen
example 93 chosen
simultaneous pedal events!
example 93 chosen
simultaneous pedal events!
example 87 chosen
example 87 chosen
example 26 chosen
simultaneous pedal events!
simultaneous pedal events!
simultaneous pedal events!
simultaneous pedal events!
simultaneous pedal events!
simultaneous pedal events!
example 26 chosen
simultaneous pedal events!
simultaneo

In [35]:
print(zp_latent.shape)

print(decoder.inputs)
print([a[0] for a in in_dict.items()])
print([a[1].shape for a in in_dict.items()])
decoder.summary()

(64, 10)
[<tf.Tensor 'z_4:0' shape=(None, 10) dtype=float32>, <tf.Tensor 'H_ar_5:0' shape=(64, 64, 88) dtype=float32>, <tf.Tensor 'V_ar_5:0' shape=(64, 64, 88) dtype=float32>]
['H_in', 'V_mean_in', 'dummy', 'H_ar', 'V_ar', 'z']
[(64, 64, 88), (64, 1), (64, 0), (64, 64, 88), (64, 64, 88), (64, 10)]
Model: "decoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
z (InputLayer)                  [(None, 10)]         0                                            
__________________________________________________________________________________________________
initial_dense (Dense)           (None, 100)          1100        z[0][0]                          
__________________________________________________________________________________________________
repeat_vector_5 (RepeatVector)  (None, 64, 100)      0           initial_dense[0][0]      

In [34]:
zp_param[1].shape

(64, 10)

In [ ]:
a = {'z': np.zeros((64, 10)), 'H_ar': np.zeros((64, 64, 88)), 'V_ar': np.zeros((64, 64, 88))}
with tf.compat.v1.Session():
    decoder(a)[0].eval()

In [ ]:
# get training data
assert seq_length % 4 == 0, 'Sequence length must be divisible by 4'
model_datas_train, seconds = data.folder2examples('training_data/midi_train' + data_folder_prefix, sparse=True, use_base_key=use_base_key, beats_per_ex=int(seq_length / sub_beats), nth_file=nth_file, vel_cutoff=vel_cutoff, sub_beats=sub_beats)
model_datas_val, seconds = data.folder2examples('training_data/midi_val' + data_folder_prefix, sparse=True, use_base_key=use_base_key, beats_per_ex=int(seq_length / sub_beats), sub_beats=sub_beats)


In [ ]:

model_input_reqs, model_output_reqs = models.get_model_reqs(model_inputs, model_outputs)
z_list, model_inputs_tf = models.create_LSTMencoder_graph(model_input_reqs,
                                                hidden_state = hidden_state,
                                                dense_size=dense_size,
                                                latent_size=latent_size,
                                                seq_length=seq_length,
                                                recurrent_dropout=recurrent_dropout,
                                                z_activation=z_activation,
                                                variational=variational,
                                                conv=conv)
encoder = tf.keras.Model(inputs=model_inputs_tf, outputs=z_list, name=f'encoder')

if variational:
    beta_fn = exp_utils.beta_fn2(beta_rate, max_beta)
    loss_for_train, beta_cb = loss(z_list, beta_fn, free_bits=free_bits, kl_weight=kl_weight)
    sampling_fn = models.sampling(batch_size, epsilon_std=epsilon_std)
    z_placeholder = layers.Lambda(sampling_fn)(z_list)
else:
    loss_for_train = loss

z_param_tf = tf.keras.Input(shape=(2,latent_size), name='z')
z_in_through_sfn = sampling_fn(z_param_tf)
z_in = tf.keras.Input(shape=(latent_size,), name='z')
pred, ar_inputs_tf = models.create_LSTMdecoder_graph_ar(z_in_through_sfn,
                                                        model_output_reqs,
                                                        seq_length=seq_length,
                                                        hidden_state=hidden_state,
                                                        dense_size=dense_size,
                                                        ar_inputs=ar_inputs,
                                                        recurrent_dropout=recurrent_dropout,
                                                        initial_state_from_dense=initial_state_from_dense,
                                                        initial_state_activation=initial_state_activation,
                                                        batch_size=batch_size,
                                                        ar_inc_batch_shape=ar_inc_batch_shape)
decoder = tf.keras.Model(inputs=[z_in] + ar_inputs_tf, outputs=pred, name=f'decoder')

pred2 = decoder(z_placeholder)
autoencoder = tf.keras.Model(inputs=model_inputs_tf + ar_inputs_tf, outputs=pred2, name=f'autoencoder')
autoencoder.summary()

# if continue_run != None:
#     autoencoder.load_weights(f'experiments/run_{continue_run}/{continue_run}_best_train_weights.hdf5')



# save a plot of the model
# tf.keras.utils.plot_model(seq_model, to_file=f'{path}model_plot.png')

dg = ml_classes.ModelDataGenerator([md for md in model_datas_train.values()],
                                    [model_in.name for model_in in model_input_reqs],
                                    [model_out.name for model_out in model_output_reqs],
                                    t_force=True, batch_size = batch_size, seq_length=seq_length)

dg_val = ml_classes.ModelDataGenerator([md for md in model_datas_val.values()],
                                    [model_in.name for model_in in model_input_reqs],
                                    [model_out.name for model_out in model_output_reqs],
                                    t_force=True, batch_size = batch_size, seq_length=seq_length)

# opt = tf.keras.optimizers.Adam(learning_rate=lr, clipvalue=clipvalue)
# autoencoder.compile(optimizer=opt, loss=loss_for_train, metrics=metrics, loss_weights=loss_weights)
# history = autoencoder.fit(dg, validation_data=dg_val, epochs=epochs, verbose=2)


In [ ]:
encoder = tf.keras.Model(inputs=model_inputs_tf, outputs=z_list, name=f'encoder')
decoder = 

In [ ]:
# del encoder
# del decoder
# del pred
# del ar_inputs
# tf.keras.backend.clear_session()
# z_list, model_inputs_tf = models.create_LSTMencoder_graph(model_input_reqs,
#                                                 hidden_state = hidden_state,
#                                                 dense_size=dense_size,
#                                                 latent_size=latent_size,
#                                                 seq_length=seq_length,
#                                                 recurrent_dropout=recurrent_dropout,
#                                                 z_activation=z_activation,
#                                                 variational=variational,
#                                                 conv=conv)
encoder = tf.keras.Model(inputs=model_inputs_tf, outputs=z_list, name=f'encoder')
z_in = tf.keras.Input(shape=(latent_size,), name='z')
pred, ar_inputs_tf = models.create_LSTMdecoder_graph_ar(z_in,
                                                    model_output_reqs,
                                                    seq_length=seq_length,
                                                    hidden_state=hidden_state,
                                                    dense_size=dense_size,
                                                    ar_inputs=ar_inputs,
                                                    recurrent_dropout=recurrent_dropout,
                                                    initial_state_from_dense=initial_state_from_dense,
                                                    initial_state_activation=initial_state_activation,
                                                    batch_size=batch_size,
                                                    ar_inc_batch_shape=ar_inc_batch_shape)
decoder = tf.keras.Model(inputs=[z_in] + ar_inputs_tf, outputs=pred, name=f'decoder')
autoencoder.save_weights(f'experiments/run_{no}/{no}_best_train_weights.hdf5')
models.load_weights_safe(decoder, path + f'{no}_best_train_weights.hdf5', by_name=True)


In [ ]:
def abb(b,a,c=3, d=4, **kwargs):
    print(a)
    print(b)
    print(c)
    print(d)

kw={'b':1, 'a':2, 'c':3, 'd':4}
abb(**kw)
print(kw)
    

In [ ]:
zp_param = encoder.predict(dg.__getitem__(0)[0])
np.array(zp_param).shape

In [ ]:
zp_latent = sampling_fn(zp_param)

In [ ]:
zp_latent.shape

In [ ]:
a = autoencoder(dg.__getitem__(0)[0])

In [ ]:
a[1].name

In [ ]:
# sess = tf.Session()
in_dict = dg.__getitem__(0)[0]

with tf.compat.v1.Session():
    in_dict['z'] = zp_latent.eval()
pred = decoder.predict(in_dict)

In [ ]:
### Make some predictions ###
# load best weights
# models.load_weights_safe(autoencoder, path + f'{no}_best_train_weights.hdf5', by_name=False)
# get some random examples from the validation data
random_examples, idx = data.n_rand_examples(model_datas_val, n=batch_size)
pred = autoencoder.predict(random_examples)
# find axis that corresponds to velocity
v_index = np.where(np.array(autoencoder.output_names) == 'V_out')[0][0]
print('velocity index:', v_index)
model_datas_pred, _ = data.folder2examples('training_data/midi_val' + data_folder_prefix, sparse=False, use_base_key=use_base_key, beats_per_ex=int(seq_length / sub_beats), sub_beats=sub_beats)
model_datas = copy.deepcopy(model_datas_pred)
model_datas_pred['V'].data[idx,...] = np.array(pred)[v_index,:,:,:]
os.mkdir(path + 'midi/')
for i in idx:
    pm_original = data.examples2pm(model_datas, i, sub_beats=sub_beats)
    pm_pred = data.examples2pm(model_datas_pred, i, sub_beats=sub_beats)
    pm_original.write(path + 'midi/' + f'ex{i}original.mid')
    pm_pred.write(path + 'midi/' + f'ex{i}prediction_teacher_forced.mid')

In [ ]:
# data.n_rand_examples(model_datas_val, n=batch_size)

# pred = autoencoder.predict(dg.__getitem__(0)[0])
[len(a) for a in dg.__getitem__(0)[0].values()]
[a for a in dg.__getitem__(0)[0].keys()]
